### Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import json
from pymongo import MongoClient

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
result = requests.get('https://www.hespress.com/economie', headers=headers)
doc = BeautifulSoup(result.text, 'html.parser')

In [3]:
main_div = doc.find_all(name='div',attrs={'class':'posts-categoy'})
posts = main_div[0].find_all(name='div',attrs={'class':'card-img-top'})
articals = []
for article in posts:
    link = article.find(name='a').get('href')
    title = article.find(name='a').get('title')
    articals.append({title : link})

In [4]:
with open('hess_article_links.json', 'w', encoding='utf-8') as f:
    data = []
    for article in articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})

    with open('hess_article_links.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [5]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f:
    article_content = []
    data = json.load(f)
    for article in data:
        article_result = requests.get(article['link'], headers=headers)
        article_doc = BeautifulSoup(article_result.text, 'html.parser')
        article_content.append(article_doc.find_all(name='div',attrs={'class':'article-content'})[0].find_all(name='p'))

In [6]:
with open('hess_article_content.json', 'w', encoding='utf-8') as f:
    contents = []
    for i,content in enumerate(article_content):
        content = ''
        for p in article_content[i]:
            content += p.text
        contents.append({'title': list(data[i].values())[0], 'content': content})

    with open('hess_article_content.json', 'w', encoding='utf-8') as f:
        json.dump(contents, f, ensure_ascii=False, indent=2)

In [7]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f1:
    links = json.load(f1)
with open('hess_article_content.json', 'r', encoding='utf-8') as f2:
    art_content = json.load(f2)
client = MongoClient('mongodb://localhost:27017/')
db = client['NLP_lab1']
coll1 = db['hess_article_links']
coll2 = db['hess_article_content']
coll1.drop()
coll1.insert_many(links)
coll2.drop()
coll2.insert_many(art_content)
client.close()

### text cleaning